In [18]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_colwidth', 100)

from statsmodels.tsa.stattools import grangercausalitytests

# Global variables and paths

In [2]:
company = 'sunrun'
ticker = 'RUN'

In [3]:
path_stocks = '../download_stock_data/' + ticker + '.csv'
path_twitter_sentiment = '../../../data/team/NLTK Time Series/sentiment_nltk_fixed_final.csv'

# Import Data

## Import Stock Data

In [4]:
stock = pd.read_csv(path_stocks)
stock['Date'] = pd.to_datetime(stock['timestamp'])
stock.set_index('Date', inplace=True);
stock.drop(['timestamp'],axis=1, inplace=True)
stock_close_daily = pd.DataFrame(stock['close'])

## Import Twitter Data

In [5]:
twitter_sentiment = pd.read_csv(path_twitter_sentiment)
twitter_sentiment.drop('Unnamed: 0', axis=1, inplace=True)
twitter_sentiment['Date'] = pd.to_datetime(twitter_sentiment['CreatedAt'].str[:10], errors='coerce')
twitter_sentiment.set_index('Date',inplace=True);
twitter_sentiment.drop('CreatedAt', axis=1, inplace=True)

# Select Company

In [6]:
# select twitter_sentiment data based on company
twitter_sentiment_company = twitter_sentiment[twitter_sentiment['company'] == company]

# Create daily tweet count 

In [7]:
tweets_daily = twitter_sentiment_company.groupby(['Date']).count()
tweets_daily = pd.DataFrame(tweets_daily['compound'])
tweets_daily.rename(columns={'compound':'Daily Tweets'},inplace=True)

# Create Sentiment data

In [10]:
sentiment = twitter_sentiment[['neg','pos','compound']]
sentiment_daily = sentiment.groupby(['Date']).count()

# Merge Data

In [12]:
data = pd.merge(tweets_daily, sentiment_daily, how='left', left_on=['Date'], right_on=['Date'] )
data = pd.merge(data, stock_close_daily, how='left', left_on=['Date'], right_on=['Date'] )
data.dropna(inplace=True)
data.rename(columns={'Daily Tweets':'daily_tweets','close':'closing_price', 'compound':'compound_sentiment', 'neg':'neg_sentiment','pos':'pos_sentiment'}, inplace=True)

# Export Data 

In [13]:
# name of output csv
output = "./data_" + company + "_"  + ticker + ".csv"

In [14]:
data.to_csv(output)

In [17]:
data

,daily_tweets,neg_sentiment,pos_sentiment,compound_sentiment,closing_price
Date,,,,,
2015-08-05,536,1125,1125,1125,10.77
2015-08-06,361,1154,1154,1154,8.77
2015-08-07,125,562,562,562,10.14
2015-08-10,63,182,182,182,10.12
2015-08-11,56,199,199,199,9.79
...,...,...,...,...,...
2021-03-25,61,613,613,613,53.00
2021-03-26,80,8921,8921,8921,57.09
2021-03-29,127,6835,6835,6835,51.91


# Normalize Data

In [20]:
def normalize_daily_tweets(df):
    max_count = df['daily_tweets'].max()
    min_count = df['daily_tweets'].min()
    df['daily_tweets'] = (df['daily_tweets'] - min_count) / (max_count - min_count)
    return df


def normalize_neg_sentiment(df):
    max_count = df['neg_sentiment'].max()
    min_count = df['neg_sentiment'].min()
    df['neg_sentiment'] = (df['neg_sentiment'] - min_count) / (max_count - min_count)
    return df


def normalize_pos_sentiment(df):
    max_count = df['pos_sentiment'].max()
    min_count = df['pos_sentiment'].min()
    df['pos_sentiment'] = (df['pos_sentiment'] - min_count) / (max_count - min_count)
    return df


def normalize_compound_sentiment(df):
    max_count = df['compound_sentiment'].max()
    min_count = df['compound_sentiment'].min()
    df['compound_sentiment'] = (df['compound_sentiment'] - min_count) / (max_count - min_count)
    return df


def normalize_closing_price(df):
    max_count = df['closing_price'].max()
    min_count = df['closing_price'].min()
    df['closing_price'] = (df['closing_price'] - min_count) / (max_count - min_count)
    return df

In [21]:
data

,daily_tweets,neg_sentiment,pos_sentiment,compound_sentiment,closing_price
Date,,,,,
2015-08-05,536,1125,1125,1125,10.77
2015-08-06,361,1154,1154,1154,8.77
2015-08-07,125,562,562,562,10.14
2015-08-10,63,182,182,182,10.12
2015-08-11,56,199,199,199,9.79
...,...,...,...,...,...
2021-03-25,61,613,613,613,53.00
2021-03-26,80,8921,8921,8921,57.09
2021-03-29,127,6835,6835,6835,51.91


In [22]:
data_normalized = normalize_daily_tweets(data)

In [24]:
data_normalized = normalize_neg_sentiment(data)

In [25]:
data_normalized = normalize_pos_sentiment(data)

In [27]:
data_normalized = normalize_compound_sentiment(data)

In [28]:
data_normalized = normalize_closing_price(data)

In [29]:
data_normalized

,daily_tweets,neg_sentiment,pos_sentiment,compound_sentiment,closing_price
Date,,,,,
2015-08-05,0.554404,0.100760,0.100760,0.100760,0.066834
2015-08-06,0.373057,0.103481,0.103481,0.103481,0.045064
2015-08-07,0.128497,0.047941,0.047941,0.047941,0.059976
2015-08-10,0.064249,0.012290,0.012290,0.012290,0.059758
2015-08-11,0.056995,0.013885,0.013885,0.013885,0.056166
...,...,...,...,...,...
2021-03-25,0.062176,0.052725,0.052725,0.052725,0.526505
2021-03-26,0.081865,0.832161,0.832161,0.832161,0.571024
2021-03-29,0.130570,0.636457,0.636457,0.636457,0.514640


In [32]:
output_normalized = "./data_" + company + "_"  + ticker + "_normalized.csv"
data.to_csv(output_normalized)
data_normalized

,daily_tweets,neg_sentiment,pos_sentiment,compound_sentiment,closing_price
Date,,,,,
2015-08-05,0.554404,0.100760,0.100760,0.100760,0.066834
2015-08-06,0.373057,0.103481,0.103481,0.103481,0.045064
2015-08-07,0.128497,0.047941,0.047941,0.047941,0.059976
2015-08-10,0.064249,0.012290,0.012290,0.012290,0.059758
2015-08-11,0.056995,0.013885,0.013885,0.013885,0.056166
...,...,...,...,...,...
2021-03-25,0.062176,0.052725,0.052725,0.052725,0.526505
2021-03-26,0.081865,0.832161,0.832161,0.832161,0.571024
2021-03-29,0.130570,0.636457,0.636457,0.636457,0.514640


In [33]:
data

,daily_tweets,neg_sentiment,pos_sentiment,compound_sentiment,closing_price
Date,,,,,
2015-08-05,0.554404,0.100760,0.100760,0.100760,0.066834
2015-08-06,0.373057,0.103481,0.103481,0.103481,0.045064
2015-08-07,0.128497,0.047941,0.047941,0.047941,0.059976
2015-08-10,0.064249,0.012290,0.012290,0.012290,0.059758
2015-08-11,0.056995,0.013885,0.013885,0.013885,0.056166
...,...,...,...,...,...
2021-03-25,0.062176,0.052725,0.052725,0.052725,0.526505
2021-03-26,0.081865,0.832161,0.832161,0.832161,0.571024
2021-03-29,0.130570,0.636457,0.636457,0.636457,0.514640
